<a href="https://colab.research.google.com/github/priaJuli/project_thesis/blob/master/Prepare_spectral_V_thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import neccesary package

In [1]:
!pip3 install pytsmod

  Using cached SoundFile-0.10.3.post1-py2.py3-none-any.whl (21 kB)
  Attempting uninstall: soundfile
    Found existing installation: soundfile 0.12.1
    Uninstalling soundfile-0.12.1:
      Successfully uninstalled soundfile-0.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.1 requires soundfile>=0.12.1, but you have soundfile 0.10.3.post1 which is incompatible.


In [2]:
!pip install -U librosa

  Using cached soundfile-0.12.1-py2.py3-none-manylinux_2_31_x86_64.whl (1.2 MB)
  Attempting uninstall: soundfile
    Found existing installation: SoundFile 0.10.3.post1
    Uninstalling SoundFile-0.10.3.post1:
      Successfully uninstalled SoundFile-0.10.3.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytsmod 0.3.7 requires soundfile<0.11,>=0.10, but you have soundfile 0.12.1 which is incompatible.


In [3]:
!pip install -U noisereduce



In [4]:
import pandas as pd
import numpy as np

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, minmax_scale
from sklearn.decomposition import PCA

In [6]:
import librosa

import os

import math
import matplotlib.pyplot as plt
import requests
from IPython.display import Audio, display

import soundfile as sf

import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

import librosa.display

print(torch.__version__)
print(torchaudio.__version__)
print(librosa.__version__)


2.0.1+cu118
2.0.2+cu118
0.10.1


In [7]:
import logging

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# !apt-get install libsox-fmt-all libsox-dev sox
from IPython.display import Audio

import pickle

# Import data

In [10]:
SAMPLE_PATH = "/content/drive/MyDrive/new_accent_speech_archieve/"

pathsplit1 = "{}saa_split1.csv".format(SAMPLE_PATH)
pathsplit2 = "{}saa_split2.csv".format(SAMPLE_PATH)
pathsplit3 = "{}saa_split3.csv".format(SAMPLE_PATH)
pathsplit4 = "{}saa_split4.csv".format(SAMPLE_PATH)
pathsplit5 = "{}saa_split5.csv".format(SAMPLE_PATH)

pathtestsplit1 = "{}saa_test_split1.csv".format(SAMPLE_PATH)
pathtestsplit2 = "{}saa_test_split2.csv".format(SAMPLE_PATH)
pathtestsplit3 = "{}saa_test_split3.csv".format(SAMPLE_PATH)
pathtestsplit4 = "{}saa_test_split4.csv".format(SAMPLE_PATH)
pathtestsplit5 = "{}saa_test_split5.csv".format(SAMPLE_PATH)

ORISAMPLE_PATH = '{}new_speakers_as_archieve.csv'.format(SAMPLE_PATH)

PROCESSEDSAMPLE_PATH = '{}DF_Proc.csv'.format(SAMPLE_PATH)

# Load function for spectral

In [11]:
import os
if not os.path.exists("waveform"):
    os.makedirs("waveform")

def fetch_waveform(list_files, downloadable=False):
  existswav = 0
  if downloadable == True:
      for filesname in tqdm(list_files):
        files = "{}.wav".format(filesname)
        file_name = "waveform/{}".format(files)
        if os.path.isfile(file_name):
            existswav += 1
      print(existswav)


In [12]:
#@title
# EX Filtered
# filtered = ['arabic', 'dutch', 'english', 'french', 'korean', 'mandarin', 'portuguese', 'russian', 'spanish']
def get_df_by_natlang(df_unpro, filtered):
    selected = []
    for idx, datas in df_unpro.iterrows():
        if(datas['native_language'] in filtered):
            selected.append(datas)

    df_selected = pd.DataFrame(selected)
    df_selected.reset_index(drop=True, inplace=True)
    return df_selected

def get_df_by_filter_1(df_unpro):
    selected = []
    for idx, datas in df_unpro.iterrows():
        if(datas['native_language'] == "english"):
            if("usa" in datas['birth_place'].split(", ")):
              selected.append(datas)
            elif("uk" in datas['birth_place'].split(", ")):
              selected.append(datas)
            else:
              continue
        else:
          selected.append(datas)
    df_selected = pd.DataFrame(selected)
    df_selected.reset_index(drop=True, inplace=True)
    return df_selected

def get_df_by_filter_2(df_unpro):
    selected = []
    for idx, datas in df_unpro.iterrows():
        if(datas['sex'] == ' male '):
            selected.append(datas)
    print(len(selected))
    df_selected = pd.DataFrame(selected)
    df_selected.reset_index(drop=True, inplace=True)

    return df_selected

# Generate dataset

In [13]:
#@title
def cleaning_pandas(df_unpro):
    df_unpro['age'] = df_unpro['age'].replace(' ', np.nan)

    df_unpro['age'] = df_unpro['age'].astype(float)

    df_unpro = df_unpro.drop(df_unpro[df_unpro['filename'].isna()].index)

    df_unpro.loc[df_unpro['birth_place'].isna(), 'english_learning_method'] = "machine"

    df_unpro.loc[df_unpro['english_learning_method'].isna(), 'english_learning_method'] = "naturalistic"

    df_unpro.loc[df_unpro['english_learning_method'] == 'naturalisstic', 'english_learning_method'] = "naturalistic"

    df_unpro.loc[df_unpro['english_residence'].isna(), 'english_residence'] = "never"

    num_labels = len(df_unpro['native_language'].unique())

    return df_unpro

Func Dump2Upload _ create_obj_waves

In [14]:
#@title

import soundfile as sf

# import noisereduce as nr

def Dump_2_upload(data2up, filename):
    fnpath = "{}.pkl".format(filename)
    fdata = open(fnpath, "wb+")
    pickle.dump(data2up, fdata)
    fdata.close()

    file_name = fnpath
    bucket = "malaustorage"
    object_name = "datasets_images_saa/{}".format(file_name)
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
        # s3_client.download_file('malaustorage', object_name, file_name)
    except ClientError as e:
        logging.error(e)
        print(0)
    print(1)

def Dump_2_upload_drive(data2up, filename):
    FEATOBJ_PATH = "/content/drive/MyDrive/Feat Object/"
    fnpath = "{}{}.pkl".format(FEATOBJ_PATH, filename)
    fdata = open(fnpath, "wb+")
    pickle.dump(data2up, fdata)
    fdata.close()

def create_obj_waves(df_unpro, createable=False, load_from_drive=False, DF_only=False):
    wavesobj = {}
    waves2fn = []
    f22length = []
    srates = []


    file_list = df_unpro['filename']

    if createable == True:
        for datas in tqdm(file_list):
            fndata = datas
            filenames = "waveform/{}.wav".format(datas)
            wavesform, sample_rate= librosa.load(filenames)

            waveform = nr.reduce_noise(y=wavesform, sr=sample_rate, stationary=True, prop_decrease=0.9)
            waveform = wavesform

            f22length.append(float(len(waveform) / sample_rate))
            srates.append(sample_rate)
            waves2fn.append(waveform)

        df_unpro["waves"] = waves2fn
        df_unpro["length"] = f22length
        df_unpro["samplerate"] = srates

        fnpath = "{}.pkl".format("Temp_wavesobject")
        fdata = open(fnpath, "wb+")
        pickle.dump(df_unpro, fdata)
        fdata.close()
    else:
        if load_from_drive:
            if DF_only:
              SAMPLE_PATH = "/content/drive/MyDrive/new_accent_speech_archieve/"
              fnpath = "{}{}".format(SAMPLE_PATH, "DF_Proc.csv")

              df_unpro = pd.read_csv(fnpath)
            else:
              SAMPLE_PATH = "/content/drive/MyDrive/new_accent_speech_archieve/"
              fnpath = "{}{}.pkl".format(SAMPLE_PATH, "Temp_wavesobject")
              ftdata1 = open(fnpath, "rb")

              unpockt = pickle.Unpickler(ftdata1)
              df_unpro = unpockt.load()

              ftdata1.close()

        else:
            fnpath = "{}.pkl".format("Temp_wavesobject")
            ftdata1 = open(fnpath, "rb")

            unpockt = pickle.Unpickler(ftdata1)
            df_unpro = unpockt.load()

            ftdata1.close()


    return df_unpro

In [15]:
from tqdm.notebook import tqdm

# Read csv files

In [16]:
df_unpro_ori = pd.read_csv(ORISAMPLE_PATH)

df_unpro_ori = cleaning_pandas(df_unpro_ori)

In [17]:
df_unpro_ori.head()

,speaker_id,filename,birth_place,native_language,age,sex,age_of_english_onset,english_learning_method,english_residence,length_of_english_residence
0,1,afrikaans1,"virginia, south africa",afrikaans,27.0,female,9.0,academic,usa,0.50
1,2,afrikaans2,"pretoria, south africa",afrikaans,40.0,male,5.0,academic,usa,10.00
2,3,agni1,"diekabo, ivory coast",agni,25.0,male,15.0,academic,usa,1.20
3,4,albanian1,"prishtina, kosovo",albanian,19.0,male,6.0,naturalistic,usa,3.00
4,5,albanian2,"tirana, albania",albanian,33.0,male,15.0,naturalistic,usa,0.04


In [18]:
#  print(np.unique(df_unpro_ori['native_language'], return_counts=True))

In [19]:
print(np.unique(df_unpro_ori['sex'], return_counts=True))

(array([' female ', ' male '], dtype=object), array([1518, 1464]))


In [20]:
# filtered = ['english', 'arabic', 'dutch','french', 'korean', 'mandarin', 'portuguese', 'russian', 'spanish']

filtered = ['english', 'arabic', 'french', 'korean', 'mandarin', 'spanish']

df_unpro = get_df_by_natlang(df_unpro_ori, filtered)

list_files = df_unpro['filename']
# birth_place = df_unpro2['birth_place']
fetch_waveform(list_files, downloadable=False)
# fetch_waveform_PV(list_files, "2", downloadable=True)
list_files.count()

1418

In [21]:

# manas

In [22]:
# df_unpro.count()

In [23]:
# df_unpro2.count()

In [24]:
# for births in birth_place:
    # print(births)

In [25]:
# df_unpro2.loc[724]

In [26]:
# manas

# Check all data from each language

In [27]:
natlang = df_unpro['native_language']

uniqlang = np.unique(natlang, return_counts=True)

total_lang = dict(zip(uniqlang[0], uniqlang[1]))

print(total_lang)

{'arabic': 200, 'english': 650, 'french': 85, 'korean': 97, 'mandarin': 153, 'spanish': 233}


In [28]:
import random

In [29]:
filtered

['english', 'arabic', 'french', 'korean', 'mandarin', 'spanish']

In [30]:
wavesobj = create_obj_waves(df_unpro, createable=False, load_from_drive=True, DF_only=False)

In [31]:
wavesobj_proc = get_df_by_natlang(wavesobj, filtered)

In [32]:
print(wavesobj_proc.keys())

Index(['speaker_id', 'filename', 'birth_place', 'native_language', 'age',
       'sex', 'age_of_english_onset', 'english_learning_method',
       'english_residence', 'length_of_english_residence', 'length',
       'samplerate', 'waves'],
      dtype='object')


# BUILD CLASS for Converting signal into Features

3 scenarios :
1. Average duration of each languages
2. Average duration of all datas
3. Average duration of each languages and average duration of english

In [33]:
import pytsmod as tsm

from sklearn.cluster import KMeans

class Features_signal_malau:
    def __init__(self, data_fn):
        self.dur_pv1 = None
        self.dur_pv2 = None
        self.dur_pv3 = None
        self.data_fn = data_fn.copy()
        self.cols_check = ['speaker_id', 'filename', 'birth_place', 'native_language', 'age', 'sex', 'age_of_english_onset', 'english_learning_method', 'english_residence', 'length_of_english_residence', 'length', 'samplerate', 'waves']
        self.waves_data = {}
        self.data_features = {}

    def transform_phase_vovoder(self, pv_version, audio, audio_fname):
        if pv_version == 1:
            target_durs_pv = self.dur_pv1
        elif pv_version == 2:
            target_durs_pv = self.dur_pv2
        elif pv_version == 3:
            target_durs_pv = self.dur_pv3

        rates_ch = target_durs_pv[audio_fname]
        y_change  = tsm.phase_vocoder(audio, rates_ch, restore_energy=True, phase_lock=True)
        return y_change

    def __validate_data_frame(self, data_fn, datas_audio):
        if len(data_fn) == len(datas_audio) and datas_audio.keys() == data_fn['filename']:
            return True
        else:
            return False

    def fit_transform(self, datas_audio=None, pv_version=None, sample_rate=22050, n_fft=5512, hop_length=2756):
        data_class = []
        idxc = 1;
        data_features = {};

        fntest = self.data_fn['filename']

        pv_version = pv_version if pv_version is not None else 1

        datas_audio = datas_audio if datas_audio is not None else self.waves_data

        # self.waves_data = datas_audio

        rata_total = int(self.data_fn['length'].mean())
        rata2 = self.data_fn.groupby(['native_language'])['length'].mean()
        self.generate_target_duration_pv()

        for idfn, datafn in tqdm(self.data_fn.iterrows()):
            fname = datafn['filename']
            sample_rate = datafn['samplerate']
            native_lang = datafn['native_language']
            waveformnew = self.transform_phase_vovoder(pv_version, datas_audio[fname], fname)
            fftsize = int(sample_rate  * 0.025)
            # fftsize = 512
            one_feats = []
            ## spectral centroid
            cens = self.specs_centroid(waveformnew, sr=sample_rate, fftsize=fftsize, hop_length=None)

            sbw = self.specs_bandw(waveformnew, sr=sample_rate,fftsize=fftsize, hop_length=None)

            contrast = self.specs_con(waveformnew, sr=sample_rate,fftsize=fftsize, hop_length=None)

            rolloff= self.specs_rolloff(waveformnew, sr=sample_rate,fftsize=fftsize, hop_length=None)

            features = []
            features.append(cens)
            features.append(sbw)
            features.append(contrast)
            features.append(rolloff)
            data_features[fname] = features

        self.data_features = data_features
        self.data_fn.drop(columns=['waves'], inplace=True)
        return data_features

    def generate_target_duration_pv(self):
        dur_pv1 = {}
        dur_pv2 = {}
        dur_pv3 = {}

        # Get average duration of all datas
        rata_total = int(self.data_fn['length'].mean())

        # Get average duration of each native languages data
        rata2 = self.data_fn.groupby(['native_language'])['length'].mean()

        # Get average duration of english native languages
        dur_EN = round(rata2['english'])

        # Get duration for pv3
        target_durs_pv3 = {}
        for onelang in self.data_fn['native_language'].unique():
          one_durs_lang = round(rata2[onelang])
          target_durs_pv3[onelang] = round((one_durs_lang+dur_EN) / 2)

        # Iteration to generate target duration of each filename
        for idx, datas in tqdm(self.data_fn.iterrows()):
            one_waves = datas['waves']
            natlan = datas['native_language']
            one_durs = datas['length']
            one_sr = datas['samplerate']
            rates_ch1 = round(round(rata2[natlan]) / (one_durs), 3)
            rates_ch2 = round(rata_total / (one_durs), 3)
            rates_ch3 = round(target_durs_pv3[natlan] / (one_durs), 3)

            dur_pv1[datas['filename']] = rates_ch1
            dur_pv2[datas['filename']] = rates_ch2
            dur_pv3[datas['filename']] = rates_ch3

        # save to variable of this class
        self.dur_pv1 = dur_pv1
        self.dur_pv2 = dur_pv2
        self.dur_pv3 = dur_pv3

    def specs_centroid(self,waveform, sr=None, fftsize=None, hop_length=None):
        n_fft=fftsize if fftsize is not None else int(sr  * 0.025)
        hop_length= hop_length if hop_length is not None else int(fftsize * 0.4)

        centroid = librosa.feature.spectral_centroid(y=waveform, sr=sr, n_fft=n_fft, hop_length=hop_length)
        return centroid

    def specs_bandw(self,waveform, sr=None, fftsize=None, hop_length=None):
        n_fft=fftsize if fftsize is not None else int(sr  * 0.025)
        hop_length= hop_length if hop_length is not None else int(fftsize * 0.4)

        sbw = librosa.feature.spectral_bandwidth(y=waveform, sr=sr, n_fft=n_fft, hop_length=hop_length)
        return sbw

    def specs_con(self, waveform, sr=None, fftsize=None, hop_length=None):
        n_fft=fftsize if fftsize is not None else int(sr  * 0.025)
        hop_length= hop_length if hop_length is not None else int(fftsize * 0.4)
        D = np.abs(librosa.stft(waveform, n_fft=n_fft, hop_length=hop_length))
        contrast = librosa.feature.spectral_contrast(S=D, sr=sr, fmin=100, n_bands=5)
        return contrast

    def specs_rolloff(self, waveform, sr=None, fftsize=None, hop_length=None):
        n_fft=fftsize if fftsize is not None else int(sr  * 0.025)
        hop_length= hop_length if hop_length is not None else int(fftsize * 0.4)

        specs_high = librosa.feature.spectral_rolloff(y=waveform, sr=sr, roll_percent=0.95, n_fft=n_fft, hop_length=hop_length)
        specs_mins = librosa.feature.spectral_rolloff(y=waveform, sr=sr, roll_percent=0.05, n_fft=n_fft, hop_length=hop_length)
        res = np.array([np.array(specs_high), np.array(specs_mins)])
        return res
    def generate_random_sample_of_feats(self, features, number_sample=2300):
        if number_sample > 2300:
            raise Exception("Number of sample must be below 2300")

        for onefiles in features:
          idx = 0
          for anas in features[onefiles]:
              flatted_feats = np.ravel(anas)
              max_start = len(flatted_feats) - number_sample
              start_point = np.random.randint(0, high=max_start)
              anas = flatted_feats[start_point:(start_point+number_sample)]
              features[onefiles][idx] = anas
              idx += 1
        return features

# Hidden code

In [34]:
wavesobj_proc.keys()

Index(['speaker_id', 'filename', 'birth_place', 'native_language', 'age',
       'sex', 'age_of_english_onset', 'english_learning_method',
       'english_residence', 'length_of_english_residence', 'length',
       'samplerate', 'waves'],
      dtype='object')

In [35]:
waves_files = {}
for idfn, datafn in tqdm(wavesobj_proc.iterrows()):
  fname = datafn['filename']
  waves_files[fname] = datafn['waves']


0it [00:00, ?it/s]

# Initialize class

In [36]:
FeatClass = Features_signal_malau(wavesobj_proc)

In [37]:
data_features = FeatClass.fit_transform(waves_files, 1)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [38]:
counts = 0
_min, _max = float('inf'), -float('inf')
for onefiles in data_features:
    for anas in data_features[onefiles]:
      _min = min(_min, anas.shape[1])
      _max = max(_max, anas.shape[1])
      break

In [39]:
_min

2304

In [40]:
_max

3412

In [41]:
random_feats = FeatClass.generate_random_sample_of_feats(data_features, number_sample=1000)

In [42]:
counts = 0
_min, _max = float('inf'), -float('inf')
for onefiles in random_feats:
    for anas in random_feats[onefiles]:
      # print(anas)
      _min = min(_min, len(anas))
      _max = max(_max, len(anas))
      break
    # break

In [43]:
print(_min, _max)

1000 1000


In [47]:
FeatClass.data_fn.keys()

Index(['speaker_id', 'filename', 'birth_place', 'native_language', 'age',
       'sex', 'age_of_english_onset', 'english_learning_method',
       'english_residence', 'length_of_english_residence', 'length',
       'samplerate'],
      dtype='object')

In [56]:
FEATOBJ_PATH = ""
fnpath = "{}Class_feats_malau.pkl".format(FEATOBJ_PATH)

In [57]:
import pickle

fdata = open(fnpath, "wb+")
pickle.dump(FeatClass, fdata)
fdata.close()

# Others incompleted code